In [1]:
import ee
import geemap
import pandas as pd
from datetime import datetime

In [43]:
geemap.ee_initialize()
oeel = geemap.requireJS()
ee.Authenticate()
ee.Initialize(project='ee-amazyar-test1')

start_date = ee.Date("2024-06-01") # с 1990 тепловая информация Landsat-5 без колебаний
end_date = ee.Date("2024-09-01")

baikal_shape = ee.FeatureCollection('projects/ee-amazyar-test/assets/baikal')


In [30]:
center = ee.Geometry.Point([105.50893757981774,51.98951989553487])
# near = ee.Geometry.Point([105.46773884934899,51.93536364377326])
# far = ee.Geometry.Point([105.46087239427087,51.88877082838407])
# inner_border = ee.Geometry.Point([105.42379353684899,51.84552337625241])
border = ee.Geometry.Point([105.40594075364587,51.82685477335608])
# outer_near = ee.Geometry.Point([105.39770100755212,51.784397341560926])
outer_to_center = ee.Geometry.Point([105.37572835130212,51.74445088742137])
outer_to_shore = ee.Geometry.Point([105.39770100755212,52.01065691259747])

In [3]:
LandsatLST = geemap.requireJS("../baikal_vkr_init/Landsat_LST.js")

In [31]:
use_ndvi = False

def get_landsat_collection(satellite_name: str):
   coll = LandsatLST.collection(satellite_name, start_date, end_date, center, use_ndvi)
   coll = coll.filterBounds(border).filterBounds(outer_to_center).filterBounds(outer_to_shore)
   return coll.select('LST')

def get_time_LST_sob(image: ee.Image):
  datetime_str = image.getString('DATE_ACQUIRED').cat(" ").cat(image.getString('SCENE_CENTER_TIME'))
  datetime_str = datetime_str.slice(0, 18)
  return ee.Date.parse('yyyy-MM-dd HH:mm:ss', datetime_str, "Asia/Irkutsk").format("yyyy-MM-dd_HH-mm-ss", "Asia/Irkutsk").getInfo()

In [32]:
landsat = ee.ImageCollection.merge(get_landsat_collection('L8'),get_landsat_collection('L9'))
landsat = ee.ImageCollection.merge(landsat,get_landsat_collection('L7'))
landsat = ee.ImageCollection.merge(landsat,get_landsat_collection('L5'))

Отображение LST в RGB

In [8]:
cmap = ['blue', 'cyan', 'green', 'yellow', 'red']

LST_sob_vis = {
   'min': 8,
   'max': 20,
   'bands': ['LST'],
   'palette': cmap
}

In [10]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [34]:
total_image_n = landsat.size().getInfo()
landsat_list = landsat.toList(total_image_n)
total_image_n

21

In [ ]:
for image_n in range(0,total_image_n):
   image = ee.Image(landsat_list.get(image_n))
   Map.addLayer(ee.Image(image).clip(baikal_shape), LST_sob_vis, str(get_time_LST_sob(image)))

Значения температуры

In [38]:
# upwelling_df = pd.DataFrame(columns=["Date", "center", "near", "far", "inner_border", "outer_border", "outer_near", "outer_far", "outer_shore"])
upwelling_df = pd.DataFrame(columns=["Date", "center", "border", "outer_to_center", "outer_to_shore"])

for image_n in range(0,total_image_n):
   image = ee.Image(landsat_list.get(image_n))
   # Map.addLayer(ee.Image(image).clip(baikal_shape), LST_sob_vis, str(get_time_LST_sob(image)))
   date = str(get_time_LST_sob(image))
   center_temp = image.reduceRegion(
         reducer=ee.Reducer.mean(),
         geometry=center
      ).getInfo()['LST']
   border_temp = image.reduceRegion(
         reducer=ee.Reducer.mean(),
         geometry=border,
         maxPixels=35000000
      ).getInfo()['LST']
   outer_to_center_temp = image.reduceRegion(
         reducer=ee.Reducer.mean(),
         geometry=outer_to_center,
         maxPixels=35000000
      ).getInfo()['LST']
   outer_to_shore_temp = image.reduceRegion(
         reducer=ee.Reducer.mean(),
         geometry=outer_to_shore,
         maxPixels=35000000
      ).getInfo()['LST']

   upwelling_df.loc[image_n] = [date, 
                                 center_temp, 
                                 border_temp, 
                                 outer_to_center_temp,
                                 outer_to_shore_temp]

upwelling_df = upwelling_df[upwelling_df['center'] > 0]
upwelling_df = upwelling_df.sort_values("Date")
upwelling_df


,Date,center,border,outer_to_center,outer_to_shore
0,2024-06-01_03-51-01,4.520500,1.951498,2.561332,4.002476
6,2024-06-08_03-57-02,3.321633,4.095477,3.541550,3.681542
11,2024-06-09_03-51-02,4.792011,3.756705,4.079714,4.229996
1,2024-06-17_03-51-01,5.224757,5.850003,5.720397,2.777523
7,2024-06-24_03-57-03,5.774143,-5.896985,-10.437600,6.457050
12,2024-06-25_03-51-01,7.813267,7.668629,7.638352,6.997372
17,2024-07-02_03-57-03,8.203812,10.102731,11.832303,9.407754
8,2024-07-10_03-57-04,13.190748,6.763571,-74.970193,14.137582
13,2024-07-11_03-51-03,14.575747,11.072802,11.854462,-0.208847
18,2024-07-18_03-57-04,13.297364,12.726604,13.661868,13.654568


In [37]:
upwelling_df.sort_values("Date")

,Date,center,border,outer_to_center,outer_to_shore
0,2024-06-01_03-51-01,4.520500,1.951498,2.561332,4.002476
6,2024-06-08_03-57-02,3.321633,4.095477,3.541550,3.681542
11,2024-06-09_03-51-02,4.792011,3.756705,4.079714,4.229996
1,2024-06-17_03-51-01,5.224757,5.850003,5.720397,2.777523
7,2024-06-24_03-57-03,5.774143,-5.896985,-10.437600,6.457050
12,2024-06-25_03-51-01,7.813267,7.668629,7.638352,6.997372
17,2024-07-02_03-57-03,8.203812,10.102731,11.832303,9.407754
2,2024-07-03_03-51-02,-11.336029,-22.953146,-9.539739,-23.894347
8,2024-07-10_03-57-04,13.190748,6.763571,-74.970193,14.137582
13,2024-07-11_03-51-03,14.575747,11.072802,11.854462,-0.208847


In [40]:
image_indices_for_export = [18, 3, 4, 9, 20, 10, 16]

In [ ]:
for image_n in image_indices_for_export:
   image = ee.Image(landsat_list.get(image_n))
   # Map.addLayer(ee.Image(image).clip(baikal_shape), LST_sob_vis, str(get_time_LST_sob(image)))
   date = str(get_time_LST_sob(image))
   desc = "lst_sob_"+date
   ee.batch.Export.image.toDrive(
      image=image.clip(baikal_shape).select('LST').visualize(LST_sob_vis),
      description=desc,
      folder="upwelling_2024_aug", 
      fileNamePrefix=desc, 
      region=baikal_shape, 
      crs='EPSG:4326', 
      maxPixels=50000000,
      fileFormat="GeoTIFF")

Date center	near	far	inner_border	outer_border	outer_near	outer_far	outer_shore
0	2024-08-19_03-57-05	12.539448	13.596312	15.350255	16.984626	17.628353	18.272412	19.464076	19.498845

In [ ]:

   near_temp = image.reduceRegion(
         reducer=ee.Reducer.mean(),
         geometry=near
      ).getInfo()['LST']
   far_temp = image.reduceRegion(
         reducer=ee.Reducer.mean(),
         geometry=far,
         maxPixels=35000000
      ).getInfo()['LST']
   inner_border_temp = image.reduceRegion(
         reducer=ee.Reducer.mean(),
         geometry=inner_border,
         maxPixels=35000000
      ).getInfo()['LST']
outer_near_temp = image.reduceRegion(
         reducer=ee.Reducer.mean(),
         geometry=outer_near,
         maxPixels=35000000
      ).getInfo()['LST']